# Likelihood and uncertainty
A notebook to illustrate the relationships between the likelihood function and different uncertainty estimates.

## Authors:
**David W. Hogg** (NYU)

## License:
- Copyright 2025 the author. All code is licensed for re-use under the open-source *MIT License*.

## Notes:
- Some overlap with `basic_inference_example.ipynb`.

## To-do:
- Make all plots consistent across all noteboooks, so they are publication-ready.

## Bugs:
- Various things hard-coded.

In [ ]:
import numpy as np
import pylab as plt
from matplotlib import rcParams
import scipy.optimize as op

In [ ]:
rcParams['figure.figsize'] = [4.0, 4.0]

In [ ]:
# set default global stuff (apologies)

N = 16
p = 2
prior_bounds = np.array([[1., 2.], [0., 2. * np.pi]])
assert prior_bounds.shape == (p, 2)
true_omega = 2.13 # hard-coded global magic variable

In [ ]:
# make fake data

def expectation(ts, pars):
    amp, phi = pars
    return amp * np.cos(true_omega * ts - phi)

def make_fake_data(seed=17):
    rng = np.random.default_rng(seed)
    ts = np.sort(7. * rng.uniform(size=N))
    ivars = 0.5 + 0.5 * rng.uniform(size=N)
    truepars = np.zeros(p) + np.nan
    for i in range(p):
        truepars[i] = rng.uniform(low=prior_bounds[i,0],
                                  high=prior_bounds[i,1])
    return ts, expectation(ts, truepars) + rng.normal(size=N) / np.sqrt(ivars), ivars, truepars

In [ ]:
ts, ys, ivars, true_pars = make_fake_data()
print(ts.shape, ys.shape, true_pars)

In [ ]:
def plot(ts, ys, ivars, true_pars, ml_pars, samples, title):
    plt.errorbar(ts, ys, yerr=1./np.sqrt(ivars), fmt="ko")
    plot_ts = np.linspace(0., 7., 1000)
    if samples is not None:
        for sample in samples:
            plt.plot(plot_ts, expectation(plot_ts, sample), "r-", lw=1, alpha=0.45)
    if true_pars is not None:
        plt.plot(plot_ts, expectation(plot_ts, true_pars), "b-", lw=1, alpha=0.45)
    if ml_pars is not None:
        plt.plot(plot_ts, expectation(plot_ts, ml_pars), "r-", lw=2, alpha=0.9)
    plt.xlabel("time")
    plt.ylabel("data value")
    plt.title(title)

plot(ts, ys, ivars, true_pars, None, None, "data and true expectation")

In [ ]:
# define likelihood in terms of phase

def negative_log_likelihood(pars, ts, ys, ivars):
    return 0.5 * np.sum(ivars * (ys - expectation(ts, pars)) ** 2)

In [ ]:
res = op.minimize(negative_log_likelihood, true_pars, args=(ts, ys, ivars))
print(res)
ml_pars = np.zeros(4) + np.nan
ml_pars_covar = np.zeros((4,4)) + np.nan
if res.success:
    ml_pars = res.x
    ml_pars_covar = res.hess_inv
print(ml_pars)

In [ ]:
plot(ts, ys, ivars, true_pars, ml_pars, None, "maximum-likelihood estimate")

In [ ]:
# define functions in terms of amplitudes

def design_matrix(ts):
    return np.vstack([np.cos(true_omega * ts), np.sin(true_omega * ts)]).T

def ml_amplitudes(ts, ys, ivars):
    X = design_matrix(ts)
    return np.linalg.solve(X.T @ (ivars[:, None] * X), X.T @ (ivars * ys))

In [ ]:
# check that everyone is cool

a1, b1 = ml_amplitudes(ts, ys, ivars)
schml_pars = np.array([np.sqrt(a1 ** 2 + b1 ** 2), np.arctan2(b1, a1)])
print(schml_pars, np.allclose(ml_pars, schml_pars))

In [ ]:
"dm = 0.2\n",
-    "mlim = (-5., 15.)\n",
-    "mvec = np.arange(mlim[0] + 0.5 * dm, mlim[1], dm)\n",
-    "dv = 0.5\n",
-    "vlim = (0., 20.)\n",
-    "vvec = np.arange(vlim[0] + 0.5 * dv, vlim[1], dv)\n",
-    "print(mvec.shape, vvec.shape)\n",
-    "ms, vs = np.meshgrid(mvec, vvec)\n",
-    "lls = np.zeros_like(ms) + np.nan\n",
-    "for i in range(lls.shape[0]):\n",
-    "    for j in range(lls.shape[1]):\n",
-    "        lls[i, j] = log_likelihood((ms[i, j], vs[i, j]), data)\n",
-    "print(lls)"

In [ ]:
"mlls = np.max(lls)\n",
-    "plt.imshow(lls, interpolation=\"nearest\", extent=mlim + vlim,\n",
-    "           vmin=mlls - 10., vmax = mlls)\n",
-    "plt.xlabel(\"mean\")\n",
-    "plt.ylabel(\"variance\")"